# Agentic - Function Calling Using Bedrock Converse API


Let's start with a simple function `convert_temperature` that converts temperature from Celsius to Fahrenheit, and vice versa.

In [1]:
def convert_temperatures(temperatures: list, to_celsius: bool):
    """
    Converts a list of temperatures to either Celsius or Fahrenheit.

    Parameters:
    - temperatures: list of temperatures
    - to_celsius:   if True, converts from Fahrenheit to Celsius;
                    if False, converts from Celsius to Fahrenheit
    Returns:
    - List of converted temperatures
    """
    converted_temperatures = []
    for temp in temperatures:
        if to_celsius:
            converted_temp = (temp - 32) * 5 / 9
        else:
            converted_temp = (temp * 9 / 5) + 32
        converted_temperatures.append(converted_temp)

    return converted_temperatures

Let's run a few examples.

In [2]:
print("Converting temperatures to Celsius:")
temperatures = convert_temperatures([32, 68, 100], to_celsius=True)
print(temperatures)

print("Converting temperatures to Fahrenheit:")
temperatures = convert_temperatures([0, 20, 37], to_celsius=False)
print(temperatures)

Converting temperatures to Celsius:
[0.0, 20.0, 37.77777777777778]
Converting temperatures to Fahrenheit:
[32.0, 68.0, 98.6]


## Use Converse API to call functions
You can use the Amazon Bedrock API to provide a model with access to tools that assist in generating responses to the messages you send. This capability, known as *Function calling*.

In the following, we will use the [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html) to call the `convert_temperatures` function.

Install the dependencies


In [ ]:
%pip install -qU boto3

In [5]:
import json
import boto3

In [6]:
bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

We don't need to provide the details (body) of the function, we can just provide the name, arguments, description, and return type. The model will only need to generate the function call.

In [7]:
# Define the configuration for our tool...
convert_temperatures_spec = {
    "toolSpec": {
        "name": "convert_temperatures",
        "description": "Convert temperatures between Celsius and Fahrenheit.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "temperatures": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "List of temperatures to be converted.",
                    },
                    "to_celsius": {
                        "type": "boolean",
                        "description": (
                            "Flag indicating conversion direction. "
                            "True for Fahrenheit to Celsius, "
                            "False for Celsius to Fahrenheit."
                        ),
                    },
                },
                "required": ["temperatures", "to_celsius"],
            }
        },
    }
}

tool_config = {"tools": [convert_temperatures_spec]}

Calling bedrock with Converse API

In [8]:
def query_bedrock(query, system=[], tool_config=tool_config):
    response = bedrock.converse(
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",
        system=system,
        messages=[{"role": "user", "content": [{"text": query}]}],
        toolConfig=tool_config,
    )
    return response

In [9]:
USER_QUERY = "Convert temperatures [32, 68, 100] from Fahrenheit to Celsius?"

In [10]:
output = query_bedrock(query=USER_QUERY)
# print(output)

In [11]:
def extract_function_call(model_output):
    function_calling = next(
        (
            c["toolUse"]
            for c in model_output["output"]["message"]["content"]
            if "toolUse" in c
        ),
        None,
    )
    name = function_calling["name"]
    tool_args = function_calling["input"]

    function_call = f"{name}(**{tool_args})"
    return function_call

In [12]:
print(extract_function_call(output))

convert_temperatures(**{'temperatures': [32, 68, 100], 'to_celsius': True})


Then you can simply run this using `exec()` function. like this:

```python
exec(funtion_call)
```
> NOTE: Avoid running LLM produced code on environments other than sandboxed environments.

In [13]:
convert_temperatures(**{"temperatures": [32, 68, 100], "to_celsius": True})

[0.0, 20.0, 37.77777777777778]

## Using Python builtin docstring

We can simply use python's builtin docstring to provide the function signature and description.


In [14]:
def my_function(arg1: int = 0, arg2: str = "") -> str:
    """this is a function to concat a string to an integer
    arg1 (int): an integer argument
    arg2 (str): a string argument

    This function increases arg1 by 1 and then concatenates arg2 to it.
    """
    return str(arg1 + 1) + arg2

In [15]:
print(my_function.__name__)
print(my_function.__doc__)

my_function
this is a function to concat a string to an integer
    arg1 (int): an integer argument
    arg2 (str): a string argument

    This function increases arg1 by 1 and then concatenates arg2 to it.
    


In [16]:
import inspect

print(inspect.signature(my_function))

(arg1: int = 0, arg2: str = '') -> str


### Setup the Tool Configuration


In [17]:
import typing


def function_signature_to_spec(func):
    sig = inspect.signature(func)
    func_name = func.__name__
    func_doc = (
        func.__doc__.strip() if func.__doc__ else "No description provided."
    )

    input_schema = {"type": "object", "properties": {}, "required": []}

    for param in sig.parameters.values():
        param_name = param.name
        param_type = (
            param.annotation
            if param.annotation != inspect.Parameter.empty
            else str
        )

        # Determine the JSON type
        if param_type == list or param_type == typing.List[float]:
            json_type = {
                "type": "array",
                "items": {"type": "number"},
                "description": f"{param_name} values.",
            }
        elif param_type == int:
            json_type = {
                "type": "integer",
                "description": f"{param_name} parameter.",
            }
        elif param_type == float:
            json_type = {
                "type": "number",
                "description": f"{param_name} parameter.",
            }
        elif param_type == str:
            json_type = {
                "type": "string",
                "description": f"{param_name} parameter.",
            }
        elif param_type == bool:
            json_type = {
                "type": "boolean",
                "description": f"{param_name} parameter.",
            }
        else:
            json_type = {
                "type": "string",
                "description": f"{param_name} parameter.",
            }

        input_schema["properties"][param_name] = json_type
        input_schema["required"].append(param_name)

    tool_spec = {
        "toolSpec": {
            "name": func_name,
            "description": func_doc,
            "inputSchema": {"json": input_schema},
        }
    }

    return tool_spec

In [18]:
# Convert the function signature
function_spec = function_signature_to_spec(my_function)
tool_config = {"tools": [function_spec]}
print(json.dumps(tool_config, indent=4))

{
    "tools": [
        {
            "toolSpec": {
                "name": "my_function",
                "description": "this is a function to concat a string to an integer\n    arg1 (int): an integer argument\n    arg2 (str): a string argument\n\n    This function increases arg1 by 1 and then concatenates arg2 to it.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "arg1": {
                                "type": "integer",
                                "description": "arg1 parameter."
                            },
                            "arg2": {
                                "type": "string",
                                "description": "arg2 parameter."
                            }
                        },
                        "required": [
                            "arg1",
                            "arg2"
                        ]


In [19]:
response = query_bedrock(
    query="increase 1 and then concat it to `hello`", tool_config=tool_config
)
# print(response)

In [20]:
function_call = extract_function_call(response)
print(function_call)

my_function(**{'arg1': 1, 'arg2': 'hello'})


In [21]:
my_function(**{"arg1": 1, "arg2": "hello"})

'2hello'

Now we can use this approach to call our `convert_temperature` function.


In [22]:
function_spec = function_signature_to_spec(convert_temperatures)
tool_config = {"tools": [function_spec]}
print(json.dumps(tool_config, indent=4))

{
    "tools": [
        {
            "toolSpec": {
                "name": "convert_temperatures",
                "description": "Converts a list of temperatures to either Celsius or Fahrenheit.\n\n    Parameters:\n    - temperatures: list of temperatures\n    - to_celsius:   if True, converts from Fahrenheit to Celsius;\n                    if False, converts from Celsius to Fahrenheit\n    Returns:\n    - List of converted temperatures",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "temperatures": {
                                "type": "array",
                                "items": {
                                    "type": "number"
                                },
                                "description": "temperatures values."
                            },
                            "to_celsius": {
                                "type

In [23]:
response = query_bedrock(
    query="What is 90F degree in C?", tool_config=tool_config
)
function_call = extract_function_call(response)
print(function_call)

convert_temperatures(**{'temperatures': [90], 'to_celsius': 'True'})


In [24]:
convert_temperatures(**{"temperatures": [90], "to_celsius": True})

[32.22222222222222]